# Sending vs Opening Snaps

Assume you're given tables with information on Snapchat users, including their ages & time spent sending & opening snaps.

Write a query to obtain a breakdown of the time spent sending vs opening snaps as a percentage of total time spent on these activities grouped by age group. Round the percentage to 2 decimal places in the output.

Notes:

* Calculate the following percentages:
   * time spent sending / (time spent sending + time spent opening)
   * time spent opening / (time spent sending + time spent opening)
* To avoid integer division in percentages, multiply by 100.0 & not 100.

# Answer

This is a two table problem. This is the `activities` table:

```
CREATE TABLE activities (
	activity_id smallint,
	user_id smallint,
	activity_type varchar(4),
	time_spent numeric(4, 2),
	activity_date timestamp
);

COPY activities
FROM '/Users/jiehengyu/Desktop/DataLemur SQL Interview Questions/Snapchat - Sending vs Opening Snaps/activities.csv'
WITH (FORMAT CSV, HEADER);

SELECT * FROM activities;
```

<img src = "activities Table.png" width = "600" style = "margin:auto"/>

This is the `age_breakdown` table:

```
CREATE TABLE age_breakdown (
	user_id smallint,
	age_bucket varchar(5)
);

INSERT INTO age_breakdown
VALUES (123, '31-35'),
	   (456, '26-30'),
	   (789, '21-25');

SELECT * FROM age_breakdown;
```

<img src = "age_breakdown Table.png" width = "600" style = "margin:auto"/>

First, I'll join the `age_breakdown` table into the `activities` table.

```
SELECT *
FROM activities
LEFT JOIN age_breakdown
	ON activities.user_id = age_breakdown.user_id;
```

<img src = "Joined activities & age_breakdown Tables.png" width = "600" style = "margin:auto"/>

Ok, now I have a better idea of what I'm working with.

```
SELECT age_bucket,
       round(sum(CASE WHEN activity_type = 'send' 
	   	   THEN time_spent ELSE NULL END) /
		   sum(CASE WHEN activity_type IN ('send', 'open')
		   THEN time_spent ELSE NULL END) * 100.0, 2)
		   AS send_pct,
	   round(sum(CASE WHEN activity_type = 'open' 
	   	   THEN time_spent ELSE NULL END) /
		   sum(CASE WHEN activity_type IN ('send', 'open')
		   THEN time_spent ELSE NULL END) * 100.0, 2)
		   AS open_pct
FROM activities
LEFT JOIN age_breakdown
	ON activities.user_id = age_breakdown.user_id
GROUP BY age_bucket
ORDER BY age_bucket;
```

<img src = "Snapchat Send & Open Percentages.png" width = "600" style = "margin:auto"/>

For the age group 21-25, the snap-sending rate is 54.31% & snap-opening rate is 45.69%. For the age group 26-30, the snap-sending rate is 82.26% & snap-opening rate is 17.74%. For the 31-35 age group, the snap-sending rate is 37.84% & the snap-opening rate is 62.16%.